# NiN

- 全连接的问题
- -卷积层需要较少的参数$c_i \times c_o \times k^2$
- 但卷积层后的第一个全连接层的参数
  - LeNet   16x5x5x120=48k
  - AlexNet 256x5x5x4096=26M
  - VGG  512x7x7x4096=102M

## NiN块
- 一个卷积层后跟两个全连接层
  - 步幅1，无填充，输出形状跟卷积层输出一样
  - 起到全连接层的作用

## NiN架构
- 无全连接层
- 交替使用NiN块和步幅为2的最大池化层
  - 逐步减少高宽和增大通道数
- 最后使用全局平均池化层得到输出
  - 其输入通道数是类别数

## 总结
- NiN块使用卷积层加两个$1\times 1$卷积层，
  - 后者对每个像素增加了非线性性
- NiN使用全局平局池化层来代替VGG和AlexNet中的全连接层
  - 不容易过拟合，更少的参数个数

In [ ]:
import torch
from torch import nn
import d2l_source.d2l as d2l

# NiN块
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

NiN

In [ ]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

查看每个块的输出形状

In [ ]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

训练

In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())